In [97]:
import json

import pandas as pd 
from prophet import Prophet


In [98]:
ori_df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
ori_df.to_csv('ori.csv')

In [99]:
df = ori_df[['location','total_deaths','date']].copy()

In [100]:
df["total_deaths"] = df["total_deaths"].fillna(0)
df.head

<bound method NDFrame.head of            location  total_deaths        date
0       Afghanistan           0.0  2020-01-05
1       Afghanistan           0.0  2020-01-06
2       Afghanistan           0.0  2020-01-07
3       Afghanistan           0.0  2020-01-08
4       Afghanistan           0.0  2020-01-09
...             ...           ...         ...
401063     Zimbabwe        5740.0  2024-05-08
401064     Zimbabwe        5740.0  2024-05-09
401065     Zimbabwe        5740.0  2024-05-10
401066     Zimbabwe        5740.0  2024-05-11
401067     Zimbabwe        5740.0  2024-05-12

[401068 rows x 3 columns]>

In [101]:
asia=df.loc[df['location'] == 'Asia'].reset_index()
africa=df.loc[df['location'] == 'Africa']
europe=df.loc[df['location'] == 'Europe']
north_america=df.loc[df['location'] == 'North America']
ocenania=df.loc[df['location'] == 'Ocenania']
south_America=df.loc[df['location'] == 'South America']

In [102]:
new_df=pd.concat([asia,africa,europe,north_america,ocenania,south_America]).reset_index().drop(columns='index')

In [103]:
new_df.tail()

,level_0,location,total_deaths,date
7987,334883,South America,1354147.0,2024-05-08
7988,334884,South America,1354147.0,2024-05-09
7989,334885,South America,1354147.0,2024-05-10
7990,334886,South America,1354147.0,2024-05-11
7991,334887,South America,1354153.0,2024-05-12


In [104]:
new_df['date']

0       2020-01-05
1       2020-01-06
2       2020-01-07
3       2020-01-08
4       2020-01-09
           ...    
7987    2024-05-08
7988    2024-05-09
7989    2024-05-10
7990    2024-05-11
7991    2024-05-12
Name: date, Length: 7992, dtype: object

In [105]:
# new_df['year']=pd.to_datetime(new_df['date']).dt.year
# new_df['month']=pd.to_datetime(new_df['date']).dt.month
# new_df['day']=pd.to_datetime(new_df['date']).dt.day

In [106]:
asia['date'] = pd.to_datetime(new_df['date'])
dt_start = pd.to_datetime('2020-01-05')
dt_end = pd.to_datetime('2024-04-28')

asia=asia[asia['date'].between(dt_start, dt_end)]

In [107]:
asia.tail()

,index,location,total_deaths,date
1571,20655,Asia,1636932.0,2024-04-24
1572,20656,Asia,1636932.0,2024-04-25
1573,20657,Asia,1636932.0,2024-04-26
1574,20658,Asia,1636932.0,2024-04-27
1575,20659,Asia,1636953.0,2024-04-28


In [108]:
df_test = pd.DataFrame()
df_test['ds'] = asia['date']
df_test['y'] = asia['total_deaths']
# df_test.set_index('ds', inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1576 entries, 0 to 1575
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1576 non-null   datetime64[ns]
 1   y       1576 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 36.9 KB


In [109]:
m = Prophet()
m.fit(df_test)
future = m.make_future_dataframe(periods=365)
future.tail()
forecast = m.predict(future)

19:30:43 - cmdstanpy - INFO - Chain [1] start processing
19:30:45 - cmdstanpy - INFO - Chain [1] done processing


In [110]:
forecast['yhat']=forecast['yhat'].astype('int') 
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1936,2025-04-24,1.660992e+06,1.130564e+06,2.182072e+06,1.141272e+06,2.188848e+06,-10313.378540,-10313.378540,-10313.378540,-1038.330747,-1038.330747,-1038.330747,-9275.047793,-9275.047793,-9275.047793,0.0,0.0,0.0,1650679
1937,2025-04-25,1.661043e+06,1.127792e+06,2.185816e+06,1.138590e+06,2.192764e+06,-11384.130544,-11384.130544,-11384.130544,-2063.666077,-2063.666077,-2063.666077,-9320.464466,-9320.464466,-9320.464466,0.0,0.0,0.0,1649658
1938,2025-04-26,1.661094e+06,1.120585e+06,2.182738e+06,1.135364e+06,2.196566e+06,-12424.988443,-12424.988443,-12424.988443,-3102.805802,-3102.805802,-3102.805802,-9322.182641,-9322.182641,-9322.182641,0.0,0.0,0.0,1648668
1939,2025-04-27,1.661144e+06,1.127440e+06,2.190561e+06,1.133927e+06,2.199027e+06,-6163.569610,-6163.569610,-6163.569610,3117.338638,3117.338638,3117.338638,-9280.908248,-9280.908248,-9280.908248,0.0,0.0,0.0,1654980
1940,2025-04-28,1.661195e+06,1.127556e+06,2.194352e+06,1.132633e+06,2.201728e+06,-7126.764943,-7126.764943,-7126.764943,2070.983249,2070.983249,2070.983249,-9197.748193,-9197.748193,-9197.748193,0.0,0.0,0.0,1654068
